<a href="https://colab.research.google.com/github/p09323028/2020f_NTU_Econometrics_I/blob/main/Textbook/CH10_Regression_with_Panel_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chapter 10: Regression with Panel Data**
Author: Jinze Wu

Student Number: p09323028

前置作業:
- import 套件
- 載入資料
- 讀取資料
- 設置變數

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

In [ ]:
!gdown --id '12vDxU458QwhxiXHL_c60InwH-DiJR7jL' --output fatality.xlsx
!gdown --id '1rG7gAPSyNigLhyavwfb9Hj9Th647ZSuG' --output fatality.pdf

In [ ]:
fatality = pd.read_excel('fatality.xlsx')

In [ ]:
fatality.year.describe()

count     336.000000
mean     1985.000000
std         2.002983
min      1982.000000
25%      1983.000000
50%      1985.000000
75%      1987.000000
max      1988.000000
Name: year, dtype: float64

In [ ]:
fatality.state.describe()

count    336.000000
mean      30.187500
std       15.309853
min        1.000000
25%       18.750000
50%       30.500000
75%       42.500000
max       56.000000
Name: state, dtype: float64

In [ ]:
# year dummies
fatality['y82'] = fatality.year.apply(lambda x: 1 if x == 1982 else 0)
fatality['y83'] = fatality.year.apply(lambda x: 1 if x == 1983 else 0)
fatality['y84'] = fatality.year.apply(lambda x: 1 if x == 1984 else 0)
fatality['y85'] = fatality.year.apply(lambda x: 1 if x == 1985 else 0)
fatality['y86'] = fatality.year.apply(lambda x: 1 if x == 1986 else 0)
fatality['y87'] = fatality.year.apply(lambda x: 1 if x == 1987 else 0)
fatality['y88'] = fatality.year.apply(lambda x: 1 if x == 1988 else 0)
# minimum legal drinking age
fatality['da18'] = fatality.mlda.apply(lambda x: 1 if x<19 else 0)
fatality['da19'] = fatality.mlda.apply(lambda x: 1 if (x>=19)&(x<20) else 0)
fatality['da20'] = fatality.mlda.apply(lambda x: 1 if (x>=20)&(x<21) else 0)
fatality['da21'] = fatality.mlda.apply(lambda x: 1 if x>21 else 0)
# fatality rate per 10,000 in the population
fatality['vfrall'] = fatality.mrall * 10000

fatality['incperc'] = fatality.perinc / 1000
fatality['lincperc'] = np.log(fatality.incperc)
fatality['vmilespd'] = fatality.vmiles / 1000
fatality['frmall'] = fatality.mrall / (fatality.vmiles / 100000)

fatality['jailcom'] = fatality.jaild + fatality.comserd
fatality['jailcom'] = fatality.jailcom.apply(lambda x: 1 if x>0 else 0)
# fatality['mjailcom'] = fatality.jaild * fatality.comserd
# fatality['jailcom'] = fatality.jailcom + fatality.mjailcom

## **10.1 Panel Data**

### Equation 10.2
$\widehat{FatalityRate} = \underset{(0.15)}{2.01} + \underset{(0.13)}{0.15} BeerTax $  (1982 data)

In [ ]:
reg_10_2 = smf.ols(formula='vfrall~beertax', data=fatality[fatality.year==1982])
results_10_2 = reg_10_2.fit(cov_type='HC1', use_t=True)
print(results_10_2.summary())

                            OLS Regression Results                            
Dep. Variable:                 vfrall   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                     1.253
Date:                Wed, 14 Apr 2021   Prob (F-statistic):              0.269
Time:                        08:17:46   Log-Likelihood:                -47.899
No. Observations:                  48   AIC:                             99.80
Df Residuals:                      46   BIC:                             103.5
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.0104      0.150     13.441      0.0

### Equation 10.3
$\widehat{FatalityRate} = \underset{(0.11)}{1.86} + \underset{(0.13)}{0.44} BeerTax $  (1988 data)

In [ ]:
reg_10_3 = smf.ols(formula='vfrall~beertax', data=fatality[fatality.year==1988])
results_10_3 = reg_10_3.fit(cov_type='HC1', use_t=True)
print(results_10_3.summary())

                            OLS Regression Results                            
Dep. Variable:                 vfrall   R-squared:                       0.134
Model:                            OLS   Adj. R-squared:                  0.115
Method:                 Least Squares   F-statistic:                     11.77
Date:                Wed, 14 Apr 2021   Prob (F-statistic):            0.00128
Time:                        08:17:46   Log-Likelihood:                -32.871
No. Observations:                  48   AIC:                             69.74
Df Residuals:                      46   BIC:                             73.49
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.8591      0.115     16.221      0.0

## **10.2 Panel Data with Two Time Periods: “Before and After” Comparisons**

In [ ]:
fatality['dvfrall'] = fatality.vfrall - fatality.vfrall.shift(6)
fatality['dbtax'] = fatality.beertax - fatality.beertax.shift(6)
fatality['dbda18'] = fatality.da18 - fatality.da18.shift(6)
fatality['dbda19'] = fatality.da19 - fatality.da19.shift(6)
fatality['dbda20'] = fatality.da20 - fatality.da20.shift(6)
fatality['dbjailcom'] = fatality.jailcom - fatality.jailcom.shift(6)
fatality['dbvmilespd'] = fatality.vmilespd - fatality.vmilespd.shift(6)
fatality['dbunrate'] = fatality.unrate - fatality.unrate.shift(6)
fatality['dblincperc'] = fatality.lincperc - fatality.lincperc.shift(6)

### Equation 10.8
$\widehat{Diff}_{fatality} = \underset{(0.065)}{-0.072} - \underset{(0.36)}{1.04} \widehat{Diff}_{beertax} $

In [ ]:
reg_10_8 = smf.ols(formula='dvfrall~dbtax', data=fatality[fatality.year==1988])
results_10_8 = reg_10_8.fit(cov_type='HC1', use_t=True)
print(results_10_8.summary())

                            OLS Regression Results                            
Dep. Variable:                dvfrall   R-squared:                       0.119
Model:                            OLS   Adj. R-squared:                  0.100
Method:                 Least Squares   F-statistic:                     8.598
Date:                Wed, 14 Apr 2021   Prob (F-statistic):            0.00523
Time:                        08:17:46   Log-Likelihood:                -22.383
No. Observations:                  48   AIC:                             48.77
Df Residuals:                      46   BIC:                             52.51
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0720      0.065     -1.102      0.2

## **10.3 Fixed Effects Regression**

### Equation 10.15
$\widehat{FatalityRate} = \underset{(0.29)}{-0.66} BeerTax + State\_Fiexed\_Effects$ 

In [ ]:
reg_10_15 = smf.ols(formula='vfrall~beertax+C(state)', data=fatality)
results_10_15 = reg_10_15.fit(cov_type='cluster', use_t=True, cov_kwds={'groups':fatality['state'], 'use_correction':False})
# print(results_10_15.summary())
results_10_15t = results_10_15.t_test('beertax=0')
print(results_10_15t.summary(xname=['beertax']))

                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
beertax       -0.6559      0.288     -2.274      0.028      -1.236      -0.076


## **10.4 Regression with Time Fixed Effects**

### Equation 10.21
$\widehat{FatalityRate} = \underset{(0.35)}{-0.64} BeerTax + State\_Fiexed\_Effects + Time\_Fiexed\_Effects$ 

In [ ]:
reg_10_21 = smf.ols(formula='vfrall~beertax+C(state)+y82+y83+y84+y85+y86+y87', data=fatality)
results_10_21 = reg_10_21.fit(cov_type='cluster', use_t=True, cov_kwds={'groups':fatality['state'], 'use_correction':False})
# print(results_10_21.summary())
results_10_21t = results_10_21.t_test('beertax=0')
print(results_10_21t.summary(xname=['beertax']))

                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
beertax       -0.6400      0.350     -1.830      0.074      -1.343       0.063


## **10.5 The Fixed Effects Regression Assumptions and Standard Errors for Fixed Effects Regression**

### Table 10.1

In [ ]:
# Column(1)
reg_t10_1_1 = smf.ols(formula='vfrall~beertax', data=fatality)
results_t10_1_1 = reg_t10_1_1.fit(cov_type='HC1', use_t=True)
results_t10_1_1t = results_t10_1_1.t_test('beertax=0')
print(results_t10_1_1t.summary(xname=['beertax']))

                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
beertax        0.3646      0.053      6.899      0.000       0.261       0.469


In [ ]:
# Column(2)
reg_t10_1_2 = smf.ols(formula='vfrall~beertax+C(state)', data=fatality)
results_t10_1_2 = reg_t10_1_2.fit(cov_type='cluster', use_t=True, cov_kwds={'groups':fatality.state, 'use_correction':False})
# print(results_t10_1_2.summary())
results_t10_1_2t = results_t10_1_2.t_test('beertax=0')
print(results_t10_1_2t.summary(xname=['beertax']))

                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
beertax       -0.6559      0.288     -2.274      0.028      -1.236      -0.076


In [ ]:
# Column(3)
reg_t10_1_3 = smf.ols(formula='vfrall~beertax+C(state)+y82+y83+y84+y85+y86+y87', data=fatality)
results_t10_1_3 = reg_t10_1_3.fit(cov_type='cluster', use_t=True, cov_kwds={'groups':fatality.state, 'use_correction':False})
# print(results_t10_1_3.summary())
results_t10_1_3t = results_t10_1_3.t_test('beertax=0')
print(results_t10_1_3t.summary(xname=['beertax']))

                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
beertax       -0.6400      0.350     -1.830      0.074      -1.343       0.063


In [ ]:
print('Time Effrcts = 0', results_t10_1_3.f_test(['y82=y83=y84=y85=y86=y87=0']))

Time Effrcts = 0 <F test: F=array([[4.40037244]]), p=0.0013165178545498433, df_denom=47, df_num=6>


In [ ]:
# Column(4)
fatality.dropna(subset=['jailcom'], inplace=True)

reg_t10_1_4 = smf.ols(formula='vfrall~beertax+C(state)+da18+da19+da20+jailcom+vmilespd+unrate+lincperc+y82+y83+y84+y85+y86+y87', data=fatality)
results_t10_1_4 = reg_t10_1_4.fit(cov_type='cluster', use_t=True, cov_kwds={'groups':fatality.state, 'use_correction':False})
# print(results_t10_1_4.summary())
results_t10_1_4t = results_t10_1_4.t_test(['beertax=0','da18=0','da19=0','da20=0','jailcom=0','vmilespd=0','unrate=0','lincperc=0'])
print(results_t10_1_4t.summary(xname=['beertax','Drinking age 18','Drinking age 19','Drinking age 20','Mandatory jail or community service?','Average vehicle miles per driver','Unemployment rate','Real income per capita (logarithm)']))

                                          Test for Constraints                                          
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
beertax                                 -0.4466      0.288     -1.552      0.127      -1.025       0.132
Drinking age 18                          0.0278      0.067      0.412      0.682      -0.108       0.163
Drinking age 19                         -0.0185      0.048     -0.383      0.703      -0.116       0.079
Drinking age 20                          0.0315      0.049      0.644      0.522      -0.067       0.130
Mandatory jail or community service?     0.0384      0.100      0.385      0.702      -0.162       0.239
Average vehicle miles per driver         0.0082      0.007      1.242      0.220      -0.005       0.022
Unemployment rate                       -0.0632      0.

In [ ]:
print('Time Effrcts = 0', results_t10_1_4.f_test(['y82=y83=y84=y85=y86=y87=0']))
print('Drinking age coefficients = 0', results_t10_1_4.f_test(['da18=da19=da20=0']))
print('Unemployment rate, income per capita = 0', results_t10_1_4.f_test(['unrate=lincperc=0']))

Time Effrcts = 0 <F test: F=array([[10.79416841]]), p=1.6052501328402644e-07, df_denom=47, df_num=6>
Drinking age coefficients = 0 <F test: F=array([[0.37648715]]), p=0.7703556566572782, df_denom=47, df_num=3>
Unemployment rate, income per capita = 0 <F test: F=array([[31.57835564]]), p=2.0278791238613143e-09, df_denom=47, df_num=2>


In [ ]:
# Column(5)
reg_t10_1_5 = smf.ols(formula='vfrall~beertax+C(state)+da18+da19+da20+jailcom+vmilespd+y82+y83+y84+y85+y86+y87', data=fatality)
results_t10_1_5 = reg_t10_1_5.fit(cov_type='cluster', use_t=True, cov_kwds={'groups':fatality.state, 'use_correction':False})
# print(results_t10_1_5.summary())
results_t10_1_5t = results_t10_1_5.t_test(['beertax=0','da18=0','da19=0','da20=0','jailcom=0','vmilespd=0'])
print(results_t10_1_5t.summary(xname=['beertax','Drinking age 18','Drinking age 19','Drinking age 20','Mandatory jail or community service?','Average vehicle miles per driver']))

                                          Test for Constraints                                          
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
beertax                                 -0.6897      0.342     -2.018      0.049      -1.377      -0.002
Drinking age 18                         -0.0102      0.080     -0.128      0.899      -0.171       0.150
Drinking age 19                         -0.0756      0.066     -1.153      0.255      -0.207       0.056
Drinking age 20                         -0.1000      0.054     -1.847      0.071      -0.209       0.009
Mandatory jail or community service?     0.0853      0.108      0.787      0.435      -0.133       0.303
Average vehicle miles per driver         0.0174      0.010      1.706      0.095      -0.003       0.038


In [ ]:
print('Time Effrcts = 0', results_t10_1_5.f_test(['y82=y83=y84=y85=y86=y87=0']))
print('Drinking age coefficients = 0', results_t10_1_5.f_test(['da18=da19=da20=0']))

Time Effrcts = 0 <F test: F=array([[3.69131516]]), p=0.004364814613814753, df_denom=47, df_num=6>
Drinking age coefficients = 0 <F test: F=array([[1.49190782]]), p=0.22892030375174618, df_denom=47, df_num=3>


In [ ]:
# Column(6)
reg_t10_1_6 = smf.ols(formula='vfrall~beertax+C(state)+mlda+jailcom+vmilespd+unrate+lincperc+y82+y83+y84+y85+y86+y87', data=fatality)
results_t10_1_6 = reg_t10_1_6.fit(cov_type='cluster', use_t=True, cov_kwds={'groups':fatality.state, 'use_correction':False})
# print(results_t10_1_6.summary())
results_t10_1_6t = results_t10_1_6.t_test(['beertax=0','mlda=0','jailcom=0','vmilespd=0','unrate=0','lincperc=0'])
print(results_t10_1_6t.summary(xname=['beertax','Drinking age','Mandatory jail or community service?','Average vehicle miles per driver','Unemployment rate','Real income per capita (logarithm)']))

                                          Test for Constraints                                          
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
beertax                                 -0.4575      0.298     -1.536      0.131      -1.057       0.142
Drinking age                            -0.0019      0.021     -0.091      0.928      -0.044       0.040
Mandatory jail or community service?     0.0391      0.100      0.390      0.698      -0.163       0.241
Average vehicle miles per driver         0.0090      0.007      1.301      0.200      -0.005       0.023
Unemployment rate                       -0.0626      0.013     -4.876      0.000      -0.088      -0.037
Real income per capita (logarithm)       1.7876      0.625      2.862      0.006       0.531       3.044


In [ ]:
print('Time Effrcts = 0', results_t10_1_6.f_test(['y82=y83=y84=y85=y86=y87=0']))
print('Unemployment rate, income per capita = 0', results_t10_1_6.f_test(['unrate=lincperc=0']))

Time Effrcts = 0 <F test: F=array([[10.89898078]]), p=1.4175903060169326e-07, df_denom=47, df_num=6>
Unemployment rate, income per capita = 0 <F test: F=array([[33.86187051]]), p=7.806333499449792e-10, df_denom=47, df_num=2>


In [ ]:
# Column(7)
did = fatality[fatality.year.isin([1982,1988])]

reg_t10_1_7 = smf.ols(formula='vfrall~beertax+C(state)+da18+da19+da20+jailcom+vmilespd+unrate+lincperc+y82', data=did)
results_t10_1_7 = reg_t10_1_7.fit(cov_type='cluster', use_t=True, cov_kwds={'groups':did.state, 'use_correction':False})
# print(results_t10_1_7.summary())
results_t10_1_7t = results_t10_1_7.t_test(['beertax=0','da18=0','da19=0','da20=0','jailcom=0','vmilespd=0','unrate=0','lincperc=0'])
print(results_t10_1_7t.summary(xname=['beertax','Drinking age 18','Drinking age 19','Drinking age 20','Mandatory jail or community service?','Average vehicle miles per driver','Unemployment rate','Real income per capita (logarithm)']))

                                          Test for Constraints                                          
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
beertax                                 -0.9243      0.320     -2.886      0.006      -1.569      -0.280
Drinking age 18                          0.0381      0.095      0.401      0.690      -0.153       0.229
Drinking age 19                         -0.0641      0.091     -0.703      0.485      -0.247       0.119
Drinking age 20                         -0.1116      0.116     -0.962      0.341      -0.345       0.122
Mandatory jail or community service?     0.0882      0.154      0.574      0.569      -0.221       0.398
Average vehicle miles per driver         0.1246      0.046      2.723      0.009       0.033       0.217
Unemployment rate                       -0.0908      0.

In [ ]:
print('Time Effrcts = 0', results_t10_1_7.f_test(['y82=0']))
print('Drinking age coefficients = 0', results_t10_1_7.f_test(['da18=da19=da20=0']))
print('Unemployment rate, income per capita = 0', results_t10_1_7.f_test(['unrate=lincperc=0']))

Time Effrcts = 0 <F test: F=array([[42.53763607]]), p=4.345611309786781e-08, df_denom=47, df_num=1>
Drinking age coefficients = 0 <F test: F=array([[0.47809789]]), p=0.6990536526400825, df_denom=47, df_num=3>
Unemployment rate, income per capita = 0 <F test: F=array([[28.3258141]]), p=8.477360285941342e-09, df_denom=47, df_num=2>


## Supplement

In [ ]:
!pip install linearmodels
import linearmodels as plm

In [ ]:
did = did.set_index(['state','year'])

In [ ]:
y = did.vfrall
x = did[['beertax','da18','da19','da20','jailcom','vmilespd','unrate','lincperc','y82']]
reg = plm.PanelOLS(dependent=y,exog=x,entity_effects=True,time_effects=True, drop_absorbed=True)
results = reg.fit(cov_type='clustered', cluster_entity=True)
print(results.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 vfrall   R-squared:                        0.6595
Estimator:                   PanelOLS   R-squared (Between):              0.8035
No. Observations:                  96   R-squared (Within):              -2.6684
Date:                Wed, Apr 14 2021   R-squared (Overall):              0.7721
Time:                        08:17:50   Log-likelihood                    67.393
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      9.4414
Entities:                          48   P-value                           0.0000
Avg Obs:                       2.0000   Distribution:                    F(8,39)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             4.1200
                            

Variables have been fully absorbed and have removed from the regression:

y82
 [model.py:1798]


In [ ]:
reg = plm.PanelOLS.from_formula('vfrall~1+beertax+da18+da19+da20+jailcom+vmilespd+unrate+lincperc+y82+EntityEffects+TimeEffects', data=did, drop_absorbed=True)
results = reg.fit(cov_type='clustered', cluster_entity=True)
print(results.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 vfrall   R-squared:                        0.6595
Estimator:                   PanelOLS   R-squared (Between):             -1.4408
No. Observations:                  96   R-squared (Within):              -2.6684
Date:                Wed, Apr 14 2021   R-squared (Overall):             -1.5886
Time:                        08:17:50   Log-likelihood                    67.393
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      9.4414
Entities:                          48   P-value                           0.0000
Avg Obs:                       2.0000   Distribution:                    F(8,39)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             4.1200
                            

Variables have been fully absorbed and have removed from the regression:

y82
 [model.py:1798]
